In [ ]:
# Send a message to Conflunet - Kafka via python


Secret key and all details
## For default Environment
=== Confluent Cloud API key ===

API key:


API secret:

Resource:

Bootstrap server:



Required connection configs for Kafka producer, consumer, and admin

bootstrap.servers=
Best practice for higher availability in librdkafka clients prior to 1.7
session.timeout.ms=45000

client.id=ccloud-p

In [ ]:
!pip install confluent-Kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 50.3 MB/s eta 0:00:00


In [ ]:
from confluent_kafka import Producer
import json
import time

In [ ]:
import pandas as pd

In [ ]:
csv_file = '/content/first_100_customers.csv'
df = pd.read_csv(csv_file)
df.head(5)

,customer_id,name,city,state,country,registration_date,is_active
0,0,Customer_0,Pune,Maharashtra,India,2023-06-29,False
1,1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True
2,2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True
3,3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False
4,4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False


In [ ]:
json_records = df.to_dict(orient='records')
json_file = 'customers.json'

with open ('json_file','w') as file:
  json.dump(json_records, file,indent=4)

print("file converted to JSON")

file converted to JSON


In [1]:
config = {
  "bootstrap.servers":"p092",
  "security.protocol":"SASL_SSL",
  "sasl.mechanisms":"PLAIN",
  "sasl.username":"",
  "sasl.password":"+G7",
  "session.timeout.ms":45000,
  "client.id":"ccloud-pd"
}

In [ ]:
producer = Producer(config)
topic = 'ecommerce'



In [ ]:
with open('json_file','r') as file:
  customers_data = json.load(file)

value = customers_data[0]
print(value)

{'customer_id': 0, 'name': 'Customer_0', 'city': 'Pune', 'state': 'Maharashtra', 'country': 'India', 'registration_date': '2023-06-29', 'is_active': False}


In [ ]:
key = value['customer_id']
print(key,value)

0 {'customer_id': 0, 'name': 'Customer_0', 'city': 'Pune', 'state': 'Maharashtra', 'country': 'India', 'registration_date': '2023-06-29', 'is_active': False}


In [ ]:
producer.produce(topic,key=key,value=value)

TypeError: a bytes-like object is required, not 'dict'

## Value is a dictionary but it wants a byte. So we have to do following steps for same.

In [ ]:
type(str(value).encode('utf-8'))

bytes

In [ ]:
producer.produce(topic,key=str(key).encode('utf-8'),value=str(value).encode('utf-8'))

## Sending multiple messages to our Kafka


In [ ]:
customers_data

[{'customer_id': 0,
  'name': 'Customer_0',
  'city': 'Pune',
  'state': 'Maharashtra',
  'country': 'India',
  'registration_date': '2023-06-29',
  'is_active': False},
 {'customer_id': 1,
  'name': 'Customer_1',
  'city': 'Bangalore',
  'state': 'Tamil Nadu',
  'country': 'India',
  'registration_date': '2023-12-07',
  'is_active': True},
 {'customer_id': 2,
  'name': 'Customer_2',
  'city': 'Hyderabad',
  'state': 'Gujarat',
  'country': 'India',
  'registration_date': '2023-10-27',
  'is_active': True},
 {'customer_id': 3,
  'name': 'Customer_3',
  'city': 'Bangalore',
  'state': 'Karnataka',
  'country': 'India',
  'registration_date': '2023-10-17',
  'is_active': False},
 {'customer_id': 4,
  'name': 'Customer_4',
  'city': 'Ahmedabad',
  'state': 'Karnataka',
  'country': 'India',
  'registration_date': '2023-03-14',
  'is_active': False},
 {'customer_id': 5,
  'name': 'Customer_5',
  'city': 'Hyderabad',
  'state': 'Karnataka',
  'country': 'India',
  'registration_date': '2023

In [ ]:
def delivery_status(err,msg):
  if(err):
    print(f"Message delivery failed : {err}")
  else:
    print(f"Message Delivered to {msg.topic()} [{msg.partition()} at offset {msg.offset()}]")


for record in customers_data:
  try:
    message_value = json.dumps(record)
    message_key = str(record['customer_id']).encode('utf-8')

    producer.produce(topic,key=message_key,value=message_value,callback=delivery_status)
    producer.poll(1)
  except Exception as e :
    print("Erro sending message : {e}")

producer.flush()  # It ensures that everything is cleared , suppose there is something enqueue, it would be cleared.
print("Message senr to kafka")

Message Delivered to ecommerce [2 at offset 5]
Message Delivered to ecommerce [2 at offset 6]
Message Delivered to ecommerce [1 at offset 0]
Message Delivered to ecommerce [1 at offset 1]
Message Delivered to ecommerce [1 at offset 2]
Message Delivered to ecommerce [1 at offset 3]
Message Delivered to ecommerce [1 at offset 4]
Message Delivered to ecommerce [0 at offset 2]
Message Delivered to ecommerce [2 at offset 7]
Message Delivered to ecommerce [0 at offset 3]
Message Delivered to ecommerce [0 at offset 4]
Message Delivered to ecommerce [0 at offset 5]
Message Delivered to ecommerce [0 at offset 6]
Message Delivered to ecommerce [2 at offset 8]
Message Delivered to ecommerce [0 at offset 7]
Message Delivered to ecommerce [1 at offset 5]
Message Delivered to ecommerce [0 at offset 8]
Message Delivered to ecommerce [2 at offset 9]
Message Delivered to ecommerce [0 at offset 9]
Message Delivered to ecommerce [1 at offset 6]
Message Delivered to ecommerce [0 at offset 10]
Message Deli